In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('C:\\Users\\M\\Machine_L\\data\\xAPI-Edu-Data.csv')
df.head()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M


## 위 데이터를 기반으로 학생의 성적을 등급을 예측하는 머신러닝 모델을 만들고 평가하시오
- 성적 Class : middle low high
- 평가지표 : accuracy, precision, recall, f1_score, roc_auc_score

In [34]:
df.Class.unique()

array(['M', 'L', 'H'], dtype=object)

In [4]:
#1. 전처리
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    480 non-null    object
 1   NationalITy               480 non-null    object
 2   PlaceofBirth              480 non-null    object
 3   StageID                   480 non-null    object
 4   GradeID                   480 non-null    object
 5   SectionID                 480 non-null    object
 6   Topic                     480 non-null    object
 7   Semester                  480 non-null    object
 8   Relation                  480 non-null    object
 9   raisedhands               480 non-null    int64 
 10  VisITedResources          480 non-null    int64 
 11  AnnouncementsView         480 non-null    int64 
 12  Discussion                480 non-null    int64 
 13  ParentAnsweringSurvey     480 non-null    object
 14  ParentschoolSatisfaction  

y가 카테고리로 되어 있을때 어떻게 할지?
- target data가 category data이면 one hot으로 바꾸지 않아도 자동으로 바꿔준다. - old version
- y를 라벨인코딩 해주면 내부에서 자동으로 one hot 변형해서 처리한다. - new version

In [6]:
#2. X, y 설정
X = df.drop('Class', axis = 1)
y = df.Class

In [7]:
# X라벨 인코딩
df_ohe = pd.get_dummies(X, dtype = 'int')

get_dummies()
- 통째로 넣어도 문자열만 바꿔주는건지?
- 지정 안해주면 object만 골라서 바꿔준다
- subset 해주면 특정 컬럼만 바꿔준다.
- 컬럼 72개면 많지는 않다.
    
### Q. 근데 그러면 안바뀐 컬럼은 object그대로라는건데 그럼 그건 삭제한다는 뜻?
아님 그거만 따로 라벨 인코딩해서 쓰나?

In [ ]:
# y라벨 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_le = le.fit_transform(y)

In [42]:
#3. train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_ohe, y_le, test_size = 0.25, random_state = 2024)

In [43]:
#4. 모델 학습
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

C:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [44]:
#5. 예측
pred = lr.predict(X_test)

In [45]:
#6. 모델평가
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.79      0.74      0.76        35
           1       0.82      0.86      0.84        36
           2       0.71      0.71      0.71        49

    accuracy                           0.77       120
   macro avg       0.77      0.77      0.77       120
weighted avg       0.77      0.77      0.77       120



In [36]:
#4. 모델학습
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [37]:
#5. 예측
pred2 = svc.predict(X_test)

In [38]:
#6. 모델 평가
from sklearn.metrics import classification_report
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

           0       0.59      0.68      0.63        25
           1       0.65      0.80      0.72        30
           2       0.60      0.44      0.51        41

    accuracy                           0.61        96
   macro avg       0.61      0.64      0.62        96
weighted avg       0.61      0.61      0.60        96



In [39]:
# hyper parameter optimization

from sklearn.model_selection import GridSearchCV

params = {'C':[0.001, 0.01, 0.1, 1, 10, 100], 'gamma':[0.00001, 0.0001, 0.01, 0.1, 1, 10, 100]} 
# => 42개(6개 * 7개)의 조합
# => 보통 가장 좋은 성과를 냈던 모델이 썼던 params 값을 모방한거임

grid = GridSearchCV(SVC(), param_grid = params, refit = True, verbose = True)
grid.fit(X_train, y_train) #자동으로 좋은 조합 찾아줌

Fitting 5 folds for each of 42 candidates, totalling 210 fits


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': [1e-05, 0.0001, 0.01, 0.1, 1, 10, 100]},
             verbose=True)

In [40]:
grid.best_params_

{'C': 100, 'gamma': 0.0001}

In [41]:
grid.best_score_

0.731784005468216

로지스틱, knn, svc 다 해보기~!

### 모델별 파라미터
- knn은 n_neighbors
- svm은 C, gamma
- 그럼 이걸 어디에 넣느냐???

### 파라미터 구한거 넣기
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid = params, refit = True, verbose = True)\
grid_svc = GridSearchCV(SVC(), param_grid = params, refit = True, verbose = True)